In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from commpy.filters import rcosfilter
from scipy import signal
from scipy.fft import fft, fftfreq

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
class FIRfilter(object):
    def __init__(self, h, Ns):
        """
        create(initialize) a new FIR filter, return inital state
        h: filter taps
        Ns: Number of samples processed per block
        """

        M = 2**(math.ceil(np.log2(Ns)+1))

        buff = np.zeros(M)

        self.h = h
        self.Ns = Ns
        self.M = M
        self.Mmask = M-1
        self.buff = buff
        self.n_t = 0 #inital tail pointer
        self.n_p = 0 #???? temp pointer
    
    def fir(self, x):
        """
        Executes FIR block
        x : samples to process
        """
        y = np.zeros(x.size)
        for i in range (x.size):
            self.buff[self.n_t] = x[i]
            self.ptr = self.n_t
            self.n_t = (self.n_t + 1) & self.Mmask # % self.M
            sum = 0.0
            for j in range((self.h).size):
                sum = sum + self.buff[self.ptr] * self.h[j]
                self.ptr = (self.ptr - 1) & self.Mmask #% self.M
            y[i] = sum
        return y

In [3]:
def raisedcosinefilter(Ntap, beta, Ts):
    """
    create raised cosine filter taps
    
    Ntap : number of taps
    Ts : symbol period

    return: filter time range, filter taps
    """

    t = np.arange(-Ntap/2,Ntap/2)
    rcfilter = np.sinc(t/Ts) * np.cos(np.pi*beta*t/Ts) / ((1 -(2*beta*t/Ts)**2)*Ts)
    return t, rcfilter

In [111]:
# Script to test the FIR filter.

# Global parameters
Nb = 100    # Number of buffers
Ns = 128	# Samples in each buffer

# Generate filter coefficients
num_taps = 64
beta = 0.1
Ts = 16

_, h_rc = raisedcosinefilter(num_taps, beta, Ts)


# Generate some random samples.
x = np.random.randn(Ns*Nb)

# simple convolution
yc = np.convolve(x, h_rc) 


# DSP like FIR filter 
state_fir = FIRfilter(h_rc, Ns)
xb = np.reshape(x, (Nb, Ns))
yb = np.zeros((Nb, Ns))
for j in range (Nb):
    yb[j] = state_fir.fir(xb[j])
yb = np.reshape(yb, Nb*Ns)

n = np.arange(x.size).reshape((Nb*Ns)) # discrete time axis for graph

# plt.plot(n,x, n, yb)
# plt.plot(yc)

fig = go.FigureWidget()
fig.update_layout(title_text="freq")
fig.add_scatter(y = abs(fft(x)[:Nb*Ns//2]), name = "original signal")
fig.add_scatter(y = abs(fft(yc)[:Nb*Ns//2]), name = "convolution with FIR")
fig.add_scatter(y = abs(fft(yb)[:Nb*Ns//2]), name = "DSP ish FIR filter")

FigureWidget({
    'data': [{'name': 'original signal',
              'type': 'scatter',
              'uid': …

##### Test the FIR filter with another input signal

In [8]:
x = np.array([1,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1]).repeat(8)

n = np.arange(x.size)
num_taps = 50
beta = 0.1
Ts = 8

_, h_rc = raisedcosinefilter(num_taps, beta, Ts)


yc = np.convolve(x, h_rc) 

state_fir = FIRfilter(h_rc, x.size)
yb = np.zeros(x.size)
yb = state_fir.fir(x)


fig = go.FigureWidget()
fig.update_layout(title_text="time")
fig.add_scatter(y = x, name = "original signal")
fig.add_scatter(y = yc, name = "convolution with FIR")
fig.add_scatter(y = yb, name = "DSP ish FIR filter")

fig


FigureWidget({
    'data': [{'name': 'original signal',
              'type': 'scatter',
              'uid': …

##### play around with filter settings

In [117]:
# Script to test the FIR filter.

# Global parameters
Nb = 100    # Number of buffers
Ns = 128	# Samples in each buffer

# Generate filter coefficients
# num_taps = 64
# beta = 0.1
# Ts = 16

# _, h_rc = raisedcosinefilter(num_taps, beta, Ts)


# Generate some random samples.
x = np.random.randn(Ns*Nb)


# plt.plot(n,x, n, yb)
# plt.plot(yc)

fig = go.FigureWidget()
fig.update_layout(title_text="freq")
fig.add_scatter(name = "original signal")
fig.add_scatter(name = "convolution")
fig.add_scatter(name = "dsp ish method")


@interact (beta = (0,1,0.1), num_taps = (64,128, 8), Ts = (1,16,1))

def update(beta = 0.1, num_taps = 64, Ts = 16):

    _, h_rc = raisedcosinefilter(num_taps, beta, Ts)
    # simple convolution
    yc = np.convolve(x, h_rc) 


    # DSP like FIR filter 
    state_fir = FIRfilter(h_rc, Ns)
    xb = np.reshape(x, (Nb, Ns))
    yb = np.zeros((Nb, Ns))
    for j in range (Nb):
        yb[j] = state_fir.fir(xb[j])
    yb = np.reshape(yb, Nb*Ns)

    n = np.arange(x.size).reshape((Nb*Ns)) # discrete time axis for graph
    with fig.batch_update():
        fig.data[0].y = abs(fft(x)[:Nb*Ns//2])
        fig.data[1].y = abs(fft(yc)[:Nb*Ns//2])
        fig.data[2].y = abs(fft(yb)[:Nb*Ns//2])
fig

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0), IntSlider(value=64, description='nu…

FigureWidget({
    'data': [{'name': 'original signal',
              'type': 'scatter',
              'uid': …

In [5]:
x = np.array([1,1,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,1]).repeat(8)

n = np.arange(x.size)

fig = go.FigureWidget()
fig.update_layout(title_text="freq")
fig.add_scatter(name = "original signal")
fig.add_scatter(name = "convolution")
fig.add_scatter(name = "dsp ish method")


@interact (beta = (0,1,0.1), num_taps = (50,128, 8), Ts = (1,16,1))

def update(beta = 0.1, num_taps = 50, Ts = 8):

    _, h_rc = raisedcosinefilter(num_taps, beta, Ts)
    # simple convolution
    yc = np.convolve(x, h_rc) 


    # DSP like FIR filter 
    state_fir = FIRfilter(h_rc, x.size)
    yb = np.zeros(x.size)
    yb = state_fir.fir(x)

    with fig.batch_update():
        fig.data[0].y = x
        fig.data[1].y = yc
        fig.data[2].y = yb
fig






interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0), IntSlider(value=50, description='nu…

FigureWidget({
    'data': [{'name': 'original signal',
              'type': 'scatter',
              'uid': …

##### Raised Cosine Filter, filter setting modification

In [94]:
fig = make_subplots(rows = 2, cols = 1, subplot_titles=("time", "freq"))
fig.add_scatter(mode = 'lines+markers',row = 1, col = 1)
fig.add_scatter(row = 2, col = 1)
fig.layout.update(showlegend=False)
figw = go.FigureWidget(fig)

@interact (beta = (0,1,0.1), num_taps = (64,128, 8), Ts = (1,16,1))

def update(beta = 0.1, num_taps = 64, Ts = 16):
    t, h_rc = raisedcosinefilter(num_taps, beta, Ts)
    with figw.batch_update():
        figw.data[0].x = t
        figw.data[0].y = h_rc
        figw.data[1].x = np.arange(h_rc.size)[:h_rc.size//2]
        figw.data[1].y = abs(fft(h_rc))[:h_rc.size//2]
figw

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0), IntSlider(value=64, description='nu…

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'type': 'scatter',
              'uid': '4…